In [1]:
!pip install --upgrade pip 
!pip install pandas
!pip install matplotlib
!pip install sklearn

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import pandas as pd
import matplotlib.pyplot as mp
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
label_encoder = LabelEncoder()

In [3]:
# Chargement CSV Train
train = pd.read_csv("titanic/data/train.csv", index_col=0)
train.tail()


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [4]:
#Trie des données du train
train = train[train['Age'].notna()]
train = train[train['Sex'].notna()]
train = train[train['Pclass'].notna()]
train = train.drop(columns=['Name', 'Fare', 'Ticket' ,'Cabin', 'Embarked'])

train['Sex']=label_encoder.fit_transform(train['Sex'])

train.tail()

,Survived,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,,
886,0,3,0,39.0,0,5
887,0,2,1,27.0,0,0
888,1,1,0,19.0,0,0
890,1,1,1,26.0,0,0
891,0,3,1,32.0,0,0


In [5]:
y = train['Survived']
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
886    0
887    0
888    1
890    1
891    0
Name: Survived, Length: 714, dtype: int64

In [6]:
X = train.drop(columns=['Survived'])
X

,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,
1,3,1,22.0,1,0
2,1,0,38.0,1,0
3,3,0,26.0,0,0
4,1,0,35.0,1,0
5,3,1,35.0,0,0
...,...,...,...,...,...
886,3,0,39.0,0,5
887,2,1,27.0,0,0
888,1,0,19.0,0,0


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
# Création du Decision Tree CLassifier
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7627118644067796
